Write a PySpark program using Jupyter notebook to analyze statistically the text files (download from drive) as follows

1. Count the number of times a word appears in the file  

2. Calculate the most common words in the file 

3. Calculate the average number of appearances of all words. 

4. Calculate the frequency of combination of two words appears in the text file. 

5. Print the two combined words that are the median in term of number of the appearances. 

Note: 

a) At the beginning of your notebook, run this “sc.getConf().getAll()” to show your configuration. 

b) Describe in your own words how you use the Spark API for this problem, explain what Spark functions do in relation to the MapReduce model. Please write your answer in the same notebook using a markdown box. 

c) Calculate the running time for each above question. 

d) Before submitting your work, stop your current notebook session, the start it again and run all from the beginning. 

Your notebook should contains cells counting from 1. 

e) You should submit 3 things (Notebook, a pdf/html of your notebook, spark master UI saved as html) in a zip file studentID.zip.

# Note
This is run on 1B.language.model.test.txt file. I first run on 1B.language.model.txt with the partition 30000 but it's out of memory in C:/, so I need to shutdown and run on test file.

import findspark library is optional. This lib is to 

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf
import re

In [3]:
conf = SparkConf().setMaster("yarn").setAppName("WordCount")

In [4]:
sc = SparkContext.getOrCreate (conf = conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
sc.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'master'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://master:8088/proxy/application_

In [6]:
'''
Function to remove punctuation and blank sentences.
'''
def removePunctuation(word):
    x = word.lower()
    return re.sub(r'[^\w\s]', '', x)

Before running on big file, I want to ensure everything works well with small one, so I create a mock RDD with 5 lines. Below is how it looks like. The detail explaination of Spark function is on the next section.

In [7]:
'''
Mock test for 5 executions. Because it's extremely expensive to process our data, I create a mock test with only 5 first lines 
'''
mock_lineRDD = sc.parallelize(['the us centers for disease control and prevention initially advised school systems to close if outbreaks occurred  then reversed itself  saying the apparent mildness of the virus meant most schools and day care centers should stay open  even if they had confirmed cases of swine flu ',
 'when ms winfrey invited suzanne somers to share her controversial views about bioidentical hormone treatment on her syndicated show in 2009  it won ms winfrey a rare dollop of unflattering press  including a newsweek cover story titled  crazy talk  oprah  wacky cures  you  ',
 'elk calling  a skill that hunters perfected long ago to lure game with the promise of a little romance  is now its own sport ',
 'don t ',
 'fish  ranked 98th in the world  fired 22 aces en route to a 63  67  5  7   76  7  4  win over seventhseeded argentinian david nalbandian '], 3)

In [8]:
'''mock 1'''
mock_mapReducedCount = mock_lineRDD.flatMap(lambda line: re.split('\s+', line.strip())).map(lambda x: (x,1)).reduceByKey(lambda a , b: a + b)
def mockCountWord(word = None):
    if (word is None) :
        print(mock_mapReducedCount.collect())
    else:
        print(mock_mapReducedCount.filter(lambda z : z[0] == word).collect())
mockCountWord('the')
mockCountWord()

[('the', 5)]


[('the', 5), ('for', 1), ('control', 1), ('prevention', 1), ('advised', 1), ('outbreaks', 1), ('reversed', 1), ('saying', 1), ('of', 4), ('most', 1), ('schools', 1), ('care', 1), ('stay', 1), ('open', 1), ('even', 1), ('they', 1), ('flu', 1), ('when', 1), ('share', 1), ('bioidentical', 1), ('hormone', 1), ('show', 1), ('it', 1), ('unflattering', 1), ('press', 1), ('including', 1), ('newsweek', 1), ('titled', 1), ('oprah', 1), ('cures', 1), ('ago', 1), ('with', 1), ('romance', 1), ('sport', 1), ('t', 1), ('ranked', 1), ('fired', 1), ('22', 1), ('aces', 1), ('route', 1), ('63', 1), ('76', 1), ('4', 1), ('win', 1), ('over', 1), ('seventhseeded', 1), ('argentinian', 1), ('nalbandian', 1), ('and', 2), ('initially', 1), ('systems', 1), ('to', 4), ('if', 2), ('itself', 1), ('apparent', 1), ('virus', 1), ('day', 1), ('had', 1), ('confirmed', 1), ('swine', 1), ('winfrey', 2), ('somers', 1), ('her', 2), ('controversial', 1), ('views', 1), ('about', 1), ('story', 1), ('crazy', 1), ('you', 1), ('e

In [9]:
'''mock 2'''
mock_most_common = mock_mapReducedCount.max(lambda x: x[1])
mock_mapReducedCount.filter(lambda z: z[1] == mock_most_common[1]).collect()

[('the', 5), ('a', 5)]

In [10]:
'''mock 3'''
mock_uniqueWordCount = mock_mapReducedCount.count()
mock_totalWordCount = (mock_mapReducedCount.map(lambda x: x[1]).reduce(lambda a, b: a+b))
mock_avg = mock_totalWordCount / mock_uniqueWordCount
mock_totalWordCount2 = mock_mapReducedCount.values().sum()
print(mock_avg)
print(mock_totalWordCount == mock_totalWordCount2)

1.184873949579832
True


In [11]:
'''mock 4'''

mock_bigrams = mock_lineRDD.map(lambda s : re.split('\s+', s.strip())).flatMap(lambda s: [((s[i],s[i+1]),1) for i in range (0, len(s)-1)])
mock_bigramCounts = mock_bigrams.reduceByKey(lambda x, y : x + y)

def mockCountBigram(word = None):
    if word is None:
        print(mock_bigramCounts.collect())
    else:
        print(mock_bigramCounts.filter(lambda z : ' '.join(z[0]) == word).collect())
mockCountBigram('to share')
mockCountBigram()

[(('to', 'share'), 1)]
[(('when', 'ms'), 1), (('suzanne', 'somers'), 1), (('somers', 'to'), 1), (('to', 'share'), 1), (('bioidentical', 'hormone'), 1), (('hormone', 'treatment'), 1), (('won', 'ms'), 1), (('a', 'rare'), 1), (('rare', 'dollop'), 1), (('including', 'a'), 1), (('a', 'newsweek'), 1), (('newsweek', 'cover'), 1), (('story', 'titled'), 1), (('titled', 'crazy'), 1), (('crazy', 'talk'), 1), (('wacky', 'cures'), 1), (('cures', 'you'), 1), (('elk', 'calling'), 1), (('calling', 'a'), 1), (('hunters', 'perfected'), 1), (('perfected', 'long'), 1), (('long', 'ago'), 1), (('lure', 'game'), 1), (('game', 'with'), 1), (('the', 'promise'), 1), (('little', 'romance'), 1), (('romance', 'is'), 1), (('own', 'sport'), 1), (('fish', 'ranked'), 1), (('ranked', '98th'), 1), (('in', 'the'), 1), (('aces', 'en'), 1), (('route', 'to'), 1), (('a', '63'), 1), (('5', '7'), 1), (('7', '76'), 1), (('david', 'nalbandian'), 1), (('the', 'us'), 1), (('centers', 'for'), 1), (('and', 'prevention'), 1), (('advi

In [12]:
'''mock 5'''
mock_countNumBigram = mock_bigramCounts.count()
mock_median_value = None

if mock_countNumBigram % 2 == 0 :
    print('even')
    mock_median_value_with_index = mock_bigramCounts.sortBy(lambda z : z[1]).zipWithIndex().filter(lambda z: z[1] == int(mock_countNumBigram/2) or z[1] == int(countNumBigram/2)-1)
else: 
    print('odd')
    mock_median_value_with_index = mock_bigramCounts.sortBy(lambda z : z[1]).zipWithIndex().filter(lambda z: z[1] == int((mock_countNumBigram - 1)/2))

mock_median_value = mock_median_value_with_index.map(lambda z: z[0]).values().mean()
mock_bigramCounts.filter(lambda z: z[1] == mock_median_value).collect()

odd


[(('the', 'us'), 1),
 (('centers', 'for'), 1),
 (('and', 'prevention'), 1),
 (('advised', 'school'), 1),
 (('school', 'systems'), 1),
 (('systems', 'to'), 1),
 (('if', 'outbreaks'), 1),
 (('then', 'reversed'), 1),
 (('reversed', 'itself'), 1),
 (('itself', 'saying'), 1),
 (('of', 'the'), 1),
 (('the', 'virus'), 1),
 (('meant', 'most'), 1),
 (('day', 'care'), 1),
 (('care', 'centers'), 1),
 (('should', 'stay'), 1),
 (('even', 'if'), 1),
 (('they', 'had'), 1),
 (('cases', 'of'), 1),
 (('swine', 'flu'), 1),
 (('when', 'ms'), 1),
 (('suzanne', 'somers'), 1),
 (('somers', 'to'), 1),
 (('to', 'share'), 1),
 (('bioidentical', 'hormone'), 1),
 (('hormone', 'treatment'), 1),
 (('won', 'ms'), 1),
 (('a', 'rare'), 1),
 (('rare', 'dollop'), 1),
 (('including', 'a'), 1),
 (('a', 'newsweek'), 1),
 (('newsweek', 'cover'), 1),
 (('story', 'titled'), 1),
 (('titled', 'crazy'), 1),
 (('crazy', 'talk'), 1),
 (('wacky', 'cures'), 1),
 (('cures', 'you'), 1),
 (('elk', 'calling'), 1),
 (('calling', 'a'), 1)

There are some useful functions in python I utilize in this exercise. 

- re.split(regex, string): split a string by a regex expression

- .strip(): remove white spaces at leading and ending positions of a string

- bool(re.match(regex, string)): check if string matches the regex expression

# 1. Count the number of times a word appears in the file

In [13]:
'''
This is to read text file in hdfs with 500 partitions.
Firstly, I remove punctuation by applying map function for RDD
Secondly, in order to remove those empty lines, I use filter function with regex check whether that string matches non-empty lines
'''

lineRDD = (sc.textFile("hdfs://master:9000/data", 500).map(lambda x: removePunctuation(x)).filter(lambda z: bool(re.match('^(?!\s*$).+', z)) is True))


'''
I apply flatMap function to lineRDD in order to flatten into a list. Each line is splitted into single words by split
funtion (of python).
In the next step, we apply mapReduce method to extract frequencies of unique words:
    - map: map every words and transform to a tuple of itself and 1
    - reduceByKey

In order to count the number of times a word appears in the file, I would like to create a function with param is a word
'''
mapReducedCount = lineRDD.flatMap(lambda line: re.split('\s+', line.strip())).map(lambda x: (x,1)).reduceByKey(lambda a , b: a + b)

def countWord(word = None):
    if (word is None) :
        print(mapReducedCount.collect())
    else:
        print(mapReducedCount.filter(lambda z : z[0] == word).collect())
        
countWord('hello')

[('hello', 124)]


Measurement from Spark UI, I got 
- Job ID: 15
- Running time: 2.7 min


This execution only contains 1 job because we only have 1 action collect here

-> Total time: 2.7 min

# 2. Calculate the most common words in the file

In [14]:
'''
So as to find the most common words in the file, I use max function.
'''
most_common = mapReducedCount.max(lambda x: x[1])
mapReducedCount.filter(lambda z: z[1] == most_common[1]).collect()

[('the', 834484)]

As measured, we have 2 jobs, respective to max() and collect():

(ID: 16, time: 55s)

(ID: 17, time: 51s)

-> Total time: 1.7 min

# 3. Calculate the average number of appearances of all words.

In [15]:
'''
In order to find average number of appearances of all words, we need to find how many distinct words in the data, and
sum of their frequency
'''
uniqueWordCount = mapReducedCount.count()
totalWordCount = (mapReducedCount.map(lambda x: x[1]).reduce(lambda a,b: a + b))
avg = totalWordCount / uniqueWordCount
print(avg)

86.71228496434169


As measured, we have 2 job, respective to count() and reduce():

(ID: 18, time: 57s)

(ID: 19, time: 51s)

-> Total time: 1.8 min

In addition to use mapReduce method to find total word counts (map every items to their value and aggregate into numeric value (sum)),
we can also use .values().sum(). 

# 4. Calculate the frequency of combination of two words appears in the text file.

For this problem, on initial lineRDD, I use map() function with tokenization lambda function by re.split(). Every items of lineRDD becomes a list of single words. After that, use flatMap() to, firstly, transform every list of lineRDD to a list of bigrams (with the help of forloop), then flatten into a list of bigrams with count value equal to 1. 
Every items has form ((word1, word2), 1)

Finally, use reduceByKey() function to get RDD of occurences of unique bigrams.

For the rest, I implement a countBigram(string) to get the counts of string. If None, it will run collect()

In [16]:
bigrams = lineRDD.map(lambda s : re.split('\s+', s.strip())).flatMap(lambda s: [((s[i],s[i+1]),1) for i in range (0, len(s)-1)])
bigramCounts = bigrams.reduceByKey(lambda x, y : x + y)

def countBigram(word = None):
    if word is None:
        print(bigramCounts.collect())
    else:
        print(bigramCounts.filter(lambda z : ' '.join(z[0]) == word).collect())
                      
countBigram('to share')

[(('to', 'share'), 320)]


(ID: 20, time: 3.7 min)

It's obvious that only one action happen here (collect).

-> Total time: 3.7 min

# 5. Print the two combined words that are the median in term of number of the appearances. 

For this problem, the idea is to find number of unique bigrams first with action count(). There are different ways to find median value but here I sort bigramCounts and transform it to a tuple with zipWithIndex(). Because using collect() and get middle index from the list is expensive, so I want to use filter() function on RDD to find middle index instead. zipWithIndex() coupled with filter() is a great combination. 

median_value is a RDD of one item (in case number of count is odd) and two items (in case number of count is even). It will be much more reasonable to check condition and find medium value but here I want to try with .values().mean() though it takes more time to run.

In [17]:
countNumBigram = bigramCounts.count()
median_value = None

if countNumBigram % 2 == 0 :
    print('Even counts of items')
    median_with_index = bigramCounts.sortBy(lambda z : z[1]).zipWithIndex().filter(lambda z: z[1] == int(countNumBigram/2) or z[1] == int(countNumBigram/2)-1)
else: 
    print('Odd counts of items')
    median_with_index = bigramCounts.sortBy(lambda z : z[1]).zipWithIndex().filter(lambda z: z[1] == int((countNumBigram - 1)/2))

median_value = median_with_index.map(lambda z: z[0]).values().mean()
print('Median of appearances in bigrams counts is {}'.format(median_value))
bigramCounts.filter(lambda z: z[1] == median_value).collect()

Odd counts of items


Median of appearances in bigrams counts is 2.0


[(('warrington', 'man'), 2),
 (('hortaosorio', 'its'), 2),
 (('coren', 'never'), 2),
 (('mostly', 'dealing'), 2),
 (('public', 'produced'), 2),
 (('pink', 'pink'), 2),
 (('vampire', 'a'), 2),
 (('beichuan', 'with'), 2),
 (('mile', 'as'), 2),
 (('security', 'test'), 2),
 (('supplying', 'child'), 2),
 (('style', 'we'), 2),
 (('domains', 'include'), 2),
 (('delayed', '20'), 2),
 (('system', 'all'), 2),
 (('backgrounds', 'having'), 2),
 (('artist', 'always'), 2),
 (('only', 'pop'), 2),
 (('kahlenberg', 'a'), 2),
 (('thrilled', 'as'), 2),
 (('sound', 'anxious'), 2),
 (('viewers', 'safely'), 2),
 (('un', 'monitoring'), 2),
 (('over', 'settlements'), 2),
 (('referred', 'him'), 2),
 (('un', 'sussex'), 2),
 (('defused', 'tensions'), 2),
 (('each', 'hotel'), 2),
 (('killers', 'terrorists'), 2),
 (('rio', 'doce'), 2),
 (('chicken', 'scraps'), 2),
 (('latvian', 'immigrant'), 2),
 (('impressive', '138'), 2),
 (('use', 'microsoft'), 2),
 (('by', 'ipod'), 2),
 (('lb', 'ashlee'), 2),
 (('suicide', 'no

The median of bigrams count is 2. 

(ID: 21, time: 46s)

(ID: 22, time: 47s)

(ID: 23, time: 48s)

(ID: 24, time: 1.5 min)

(ID: 25, time: 43s)

(ID: 26, time: 1.4 min)

-> Total time: 5.9 min

When investigating the Jobs execution, I notice one interesting thing, that is sortBy() has 2 jobs ID 22 and ID 23, and eventhough sortBy() is implemented on sortByKey() which is a transformation, not an action itself. To validate this, I run below two executions and observe Spark UI

In [18]:
bigramCounts.sortBy(lambda z : z[1]).zipWithIndex().filter(lambda z: z[1] == int((countNumBigram - 1)/2))

PythonRDD[61] at RDD at PythonRDD.scala:53

In [19]:
bigramCounts.sortBy(lambda z : z[1])

PythonRDD[68] at RDD at PythonRDD.scala:53

(ID: 27, time: 1.2 min) and (ID: 28, time: 1.1 min) are 2 consecutive jobs of sortBy(). 

(ID: 30, time: 52s) and (ID: 31, time: 50s) are 2 consecutive jobs of sortBy(). 

This sound interesting to get deep understanding of what's behind, but I will leave for future investigation.